In [1]:
from deepset_attention import *
MODEL = DeepSet(in_features=5, feats=[80,120,70,50,8], n_class=2,pool="mean") #for full dataset Mean pooling, for small dataset Max pooling
model=MODEL
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=model.to(device) 


pool:  mean
pool:  mean
pool:  mean
pool:  mean
pool:  mean
pool:  mean


In [2]:
import torch
from torch.utils.data import Dataset
import pickle
import os
import numpy as np
import random
import pandas as pd

class StreamingHcaDataset(Dataset): 
    def __init__(self, proton_dir, pion_dir, features=["x", "y", "z", "total_energy","mean_time"]):
        super().__init__()
        
        self.proton_files = sorted([os.path.join(proton_dir, f) for f in os.listdir(proton_dir) if f.endswith(".pkl")])
        self.pion_files = sorted([os.path.join(pion_dir, f) for f in os.listdir(pion_dir) if f.endswith(".pkl")])

        self.features = features
        self.all_files = self.proton_files + self.pion_files  # Combine file lists
        self.labels = [0] * len(self.proton_files) + [1] * len(self.pion_files)  # 0 for proton, 1 for pion

    def __len__(self):
        return len(self.all_files)  # Total number of files

    def _load_file(self, file_path, label):
        """Loads a single pickle file (containing a single DataFrame) and returns point cloud data with labels."""
        # with open(file_path, "rb") as f:
        #     df = pickle.load(f)  # Load single DataFrame
    
        df=pd.read_pickle(file_path)
        # df=pd.read_parquet(file_path,engine='fastparquet')
        df=df[df["total_energy"]>5]
        
          
        part_feat = df[self.features].to_numpy()

        # Handle NaN and Inf values
        part_feat[np.isnan(part_feat)] = 0.0
        part_feat[np.isinf(part_feat)] = 0.0

        return {
            "part": torch.tensor(part_feat, dtype=torch.float32),
            "label": torch.tensor(label, dtype=torch.long),
            "seq_length": torch.tensor(part_feat.shape[0], dtype=torch.long),
        }

    def __getitem__(self, idx):
        random_idx = random.randint(0, len(self.all_files) - 1)  # Pick a random file
        file_path = self.all_files[random_idx]
        label = self.labels[random_idx]

        return self._load_file(file_path, label)  # Return data from the chosen file


In [3]:
import torch
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    """Custom collate function to handle variable-length point cloud data."""
    parts = [item["part"] for item in batch]  # List of tensors (each of shape [N, 5])
    labels = torch.tensor([item["label"] for item in batch], dtype=torch.long)  # Convert list to tensor
    seq_lengths = torch.tensor([item["seq_length"] for item in batch], dtype=torch.long)  # Convert list to tensor

    # Pad variable-length tensors to the longest sequence in the batch
    padded_parts = pad_sequence(parts, batch_first=True, padding_value=0.0)  # Shape [batch_size, max_seq_len, 5]

    return {"part": padded_parts, "label": labels, "seq_length": seq_lengths}


In [4]:

energy="25"
granularity="200"
pion_dir =f"/mnt/c/Users/hnayak/Documents/{energy}GeV/small_PKL_pion_{energy}GeV_{granularity}"
proton_dir =f"/mnt/c/Users/hnayak/Documents/{energy}GeV/small_PKL_proton_{energy}GeV_{granularity}"
name=proton_dir.replace(f"/mnt/c/Users/hnayak/Documents/{energy}GeV/small_PKL_proton_","")
print(name)
from torch.utils.data import random_split, DataLoader

# Define dataset
train_dataset = StreamingHcaDataset(proton_dir=proton_dir, pion_dir=pion_dir)

# Define split sizes
total_size = len(train_dataset)
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size  # Ensure the sum matches total_size

# Split dataset
train_set, val_set, test_set = random_split(train_dataset, [train_size, val_size, test_size])

# Define DataLoaders
train_loader = DataLoader(train_set, batch_size=1, shuffle=True, collate_fn=collate_fn, num_workers=32)
val_loader = DataLoader(val_set, batch_size=1, shuffle=False, collate_fn=collate_fn,num_workers=32)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, collate_fn=collate_fn,num_workers=32)


print(f"Train: {train_size}, Validation: {val_size}, Test: {test_size}")



25GeV_200
Train: 160000, Validation: 20000, Test: 20000


In [5]:
import tqdm
def test_model(model, test_loader, criterion=None, device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.to(device)
    model.eval()  # Set model to evaluation mode

    total_loss = 0.0
    correct = 0
    total_samples = 0

    # Progress bar for testing
    test_loader_tqdm = tqdm(enumerate(test_loader), total=len(test_loader), desc="Testing")

    with torch.no_grad():
        for i,batch in test_loader_tqdm:
            parts = batch["part"].to(device)         # Input point cloud data
            labels = batch["label"].to(device)  # Labels
            batch_size,seq_len,feat_dim=parts.shape
            parts=parts.cpu().numpy().reshape(-1,feat_dim)
            qt = QuantileTransformer(output_distribution='normal', random_state=42)
            parts = qt.fit_transform(parts)
            parts=torch.tensor(parts).reshape(batch_size,seq_len,feat_dim).to(device)

            outputs = model(parts)  # Forward pass
            loss = criterion(outputs, labels) if criterion else 0  # Compute loss if provided
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)  # Get class prediction
            correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)

            test_loader_tqdm.set_postfix(loss=loss.item())  # Update progress bar

    avg_loss = total_loss / len(test_loader) if criterion else 0
    accuracy = correct / total_samples * 100

    print(f"Test Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
    return avg_loss , accuracy

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import pandas as pd
from sklearn.preprocessing import QuantileTransformer

def train_model(model, train_loader, val_loader, num_epochs=2, learning_rate=5e-4, device=None, save_path=f"./Models/Z_{name}.pth", log_path=f"./Logs/log_summary_Z_{name}.csv"):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    best_val_loss = float("inf")  # Initialize best loss

    log_data = []  # To store log info for CSV

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        train_loader_tqdm = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}")

        for i, batch in train_loader_tqdm:
            parts = batch["part"].to(device)
            batch_size, seq_len, feat_dim = parts.shape
            parts = parts.cpu().numpy().reshape(-1, feat_dim)
            qt = QuantileTransformer(output_distribution='normal', random_state=42)
            parts = qt.fit_transform(parts)
            parts = torch.tensor(parts).reshape(batch_size, seq_len, feat_dim).float().to(device)

            labels = batch["label"].to(device)

            optimizer.zero_grad()
            outputs = model(parts)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            train_loader_tqdm.set_postfix(loss=loss.item())

        avg_train_loss = running_loss / len(train_loader)
        val_loss, Accuracy = test_model(model, val_loader, criterion, device)
         

        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # Save log
        log_data.append({
            "Epoch": epoch + 1,
            "Train Loss": avg_train_loss,
            "Val Loss": val_loss,
            "Accuracy": Accuracy,  # Placeholder for accuracy
        })

        # Save model if validation improves
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), save_path)
            print(f"Model saved at epoch {epoch+1} with val loss {val_loss:.4f}")

    # Save the log to CSV
    # df_log = pd.DataFrame(log_data)
    # df_log.to_csv(log_path, index=False)
    print(f"Training log saved to {log_path}")
    print("Training complete!")


In [7]:

# if __name__ == "__main__":
#     #Train the model
#     train_model(model, train_loader,val_loader=val_loader, num_epochs=60, learning_rate=5e-4, device=device)

In [8]:
model_test = MODEL
model_name=f"./Models/Z_{name}.pth"
print(model_name)
model_test.load_state_dict(torch.load(f"./Models/Z_{name}.pth",weights_only=True))
model_test.to(device)

./Models/Z_25GeV_200.pth


DeepSet(
  (sequential): ModuleList(
    (0): DeepSetLayer(
      (Gamma): Linear(in_features=5, out_features=80, bias=True)
      (Lambda): Linear(in_features=5, out_features=80, bias=True)
      (bn): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    )
    (1): LeakyReLU(negative_slope=0.01)
    (2): DeepSetLayer(
      (Gamma): Linear(in_features=80, out_features=120, bias=True)
      (Lambda): Linear(in_features=80, out_features=120, bias=True)
      (bn): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    )
    (3): LeakyReLU(negative_slope=0.01)
    (4): DeepSetLayer(
      (Gamma): Linear(in_features=120, out_features=70, bias=True)
      (Lambda): Linear(in_features=120, out_features=70, bias=True)
      (bn): BatchNorm1d(70, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    )
    (5): LeakyReLU(negative_slope=0.01)
    (6): DeepSetLayer(
      (Gamma): Linear(in_features=70, out_features=5

In [9]:
total_params = sum(p.numel() for p in MODEL.parameters())
print(total_params)

45292


In [10]:
from sklearn.metrics import confusion_matrix, roc_curve, auc
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from scipy.stats import beta
from matplotlib.font_manager import FontProperties

bold_font = FontProperties(weight='bold', size=14)

def evaluate_model(model, data_loader, criterion, device, name1="model", return_accuracy=False):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    data_loader_tqdm = tqdm(enumerate(data_loader), desc="Testing", total=len(data_loader))
    
    true_labels = []
    pred_labels = []
    all_scores = []

    with torch.no_grad():
        for i, batch in data_loader_tqdm:
            parts = batch["part"].to(device)
            batch_size, seq_len, feat_dim = parts.shape
            parts_np = parts.cpu().numpy().reshape(-1, feat_dim)

            qt = QuantileTransformer(output_distribution='normal', random_state=42)
            parts_np = qt.fit_transform(parts_np)
            parts = torch.tensor(parts_np).reshape(batch_size, seq_len, feat_dim).to(device)

            labels = batch["label"].to(device)
            outputs = model(parts)
            preds = F.softmax(outputs, dim=1)

            # Collect soft scores for the positive class (class 1)
            all_scores.extend(preds[:, 1].detach().cpu().numpy())
            pred_labels.extend(torch.argmax(preds, dim=-1).cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

            loss = criterion(outputs, labels)
            total_loss += loss.item()

            if return_accuracy:
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

    avg_loss = total_loss / len(data_loader)
    accuracy = 100 * correct / total if return_accuracy else None

    # Confusion Matrix
    cm = confusion_matrix(true_labels, pred_labels)
    plt.figure(figsize=(10, 8))
    colors = ["#cce5ff", "#004c99"]
    cmap = LinearSegmentedColormap.from_list("Custom Blue", colors)
    sns.heatmap(cm, annot=True, fmt='d', cmap=cmap,
                xticklabels=["Proton", "Pion"], yticklabels=["Proton", "Pion"],
                annot_kws={"size": 18, "weight": "bold"})
    
    TP, FN = cm[0, 0], cm[0, 1]
    FP, TN = cm[1, 0], cm[1, 1]
    n_total = TP + TN + FP + FN
    n_correct = TP + TN
    alpha = 0.32

    lower_bound = beta.ppf(alpha / 2, n_correct, n_total - n_correct + 1)
    upper_bound = beta.ppf(1 - alpha / 2, n_correct + 1, n_total - n_correct)
    
    title = f"Accuracy: {accuracy:.2f}%, 68% CI: [{lower_bound*100:.2f}%, {upper_bound*100:.2f}%]"
    plt.title(title, fontsize=15, weight='bold')
    plt.xlabel('Predicted', size=14, weight='bold')
    plt.ylabel('True', size=14, weight='bold')
    plt.xticks(size=14, weight='bold')
    plt.yticks(size=14, weight='bold')
    print(f"./Plots/confusion_matrix_{name}.pdf")
    # plt.savefig(f"./Plots/confusion_matrix_{name}.pdf", dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Confusion Matrix:\n{cm}")
    print(f"68% Confidence Interval: [{lower_bound:.4f}, {upper_bound:.4f}]")

    # ROC Curve
    y_true = np.array(true_labels)
    y_scores = np.array(all_scores)

    # np.savez(f"./Scores/scores_{name}.npz", array1=y_true, array2=y_scores)
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    roc_auc = auc(fpr, tpr)
    jscore = tpr - fpr
    j_index = np.argmax(jscore)
    threshold = thresholds[j_index]
    print(f"Optimal Threshold: {threshold:.4f} (J-Index: {j_index})")

    N_x = sum(y_true == 0)
    N_y = sum(y_true == 1)
    sigma_fpr = np.sqrt(fpr * (1 - fpr) / N_x)
    sigma_tpr = np.sqrt(tpr * (1 - tpr) / N_y)

    plt.figure()
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f} )')
    plt.fill_between(fpr, tpr - 5 * sigma_tpr, tpr + 5 * sigma_tpr,
                     color='blue', alpha=0.25, label='1-sigma region (PiPR) [5x]')
    plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--', label='Random Guess')
    plt.xlabel('False Positive Rate', fontsize=16,weight='bold')
    plt.ylabel('True Positive Rate', fontsize=16,weight='bold')
    plt.xticks(fontsize=14,weight='bold')
    plt.yticks(fontsize=14,weight='bold')
    plt.title('Receiver Operating Characteristic (ROC)', fontsize=17,weight='bold')
    # plt.legend(loc='lower right', fontsize=14, weight='bold')
    plt.legend(title="",prop={'weight': 'bold', 'size': 14},title_fontproperties=bold_font, loc='lower right')
    plt.grid()
    # plt.savefig(f"./Plots/roc_curve_{name}.pdf")

    return (avg_loss, f"{accuracy:.2f}%") if return_accuracy else avg_loss



In [11]:
evaluate_model(model_test, test_loader, nn.CrossEntropyLoss(), device, return_accuracy=True)

Testing:   0%|          | 0/20000 [00:00<?, ?it/s]/home/alma1/anaconda3/envs/work_env/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:2764: UserWarning: n_quantiles (1000) is greater than the total number of samples (963). n_quantiles is set to n_samples.
  warnings.warn(


Set: tensor([[[ 0.0482, -0.0117, -5.1993, -0.3789, -5.1993],
         [ 0.0482,  0.2434, -3.0787, -0.4127, -3.0787],
         [ 0.0482,  0.2434, -2.8659, -0.4212, -2.8659],
         ...,
         [-0.3469, -1.4480, -0.4355, -1.4555, -0.4965],
         [-0.3469, -1.3413, -0.3677, -0.5716, -0.4212],
         [-2.4433,  1.0558,  0.2568, -1.6163,  0.2126]]], device='cuda:0') shape:  torch.Size([1, 963, 5])
Set: tensor([[[ 1.0544e+01, -2.2799e-02,  1.5721e-01,  ..., -1.2333e-01,
          -1.0967e-02,  8.8862e+00],
         [ 6.6944e+00, -5.5024e-03,  3.2012e-01,  ..., -7.9900e-02,
          -1.5694e-03,  4.7348e+00],
         [ 6.2440e+00, -4.0294e-03,  3.2688e-01,  ..., -7.5254e-02,
          -1.2708e-03,  4.3276e+00],
         ...,
         [-2.7352e-02,  1.1192e+00,  3.2934e-01,  ..., -9.7325e-04,
          -5.0089e-02, -2.3040e-03],
         [-2.7007e-02, -6.9602e-03, -6.2435e-03,  ...,  1.7644e-01,
          -4.9380e-02,  4.0005e-01],
         [ 3.3422e+00,  6.9518e+00,  1.3409e+00,  

Testing:   0%|          | 1/20000 [00:00<3:56:32,  1.41it/s]/home/alma1/anaconda3/envs/work_env/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:2764: UserWarning: n_quantiles (1000) is greater than the total number of samples (869). n_quantiles is set to n_samples.
  warnings.warn(
/home/alma1/anaconda3/envs/work_env/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:2764: UserWarning: n_quantiles (1000) is greater than the total number of samples (999). n_quantiles is set to n_samples.
  warnings.warn(
/home/alma1/anaconda3/envs/work_env/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:2764: UserWarning: n_quantiles (1000) is greater than the total number of samples (759). n_quantiles is set to n_samples.
  warnings.warn(
/home/alma1/anaconda3/envs/work_env/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:2764: UserWarning: n_quantiles (1000) is greater than the total number of samples (942). n_quantiles is set to n_samples.
  warnings.warn(


tensor([[[-2.0587e-01,  6.0141e+00,  5.7892e+00,  ..., -4.2032e-01,
          -3.8221e-01,  1.5230e+01],
         [-9.3404e-02,  4.6684e+00,  3.6298e+00,  ..., -2.4168e-01,
          -2.2773e-01,  8.0573e+00],
         [-8.4747e-02,  4.4920e+00,  3.3693e+00,  ..., -2.2225e-01,
          -2.0964e-01,  7.0910e+00],
         ...,
         [-7.9693e-02, -1.1407e-01, -5.5249e-02,  ...,  3.5826e+00,
          -2.3394e-02, -8.1025e-02],
         [-8.2516e-02, -2.3027e-02, -8.5398e-02,  ..., -2.3226e-02,
          -1.1218e-02, -8.5985e-02],
         [ 1.8096e+01, -3.1746e-01, -2.5403e-01,  ...,  9.2410e+00,
          -1.4218e-01,  4.7674e+00]]], device='cuda:0') shape:  torch.Size([1, 963, 120])
Set: tensor([[[-6.2695e-01,  3.3734e+01, -1.6909e+00,  ...,  7.8009e+01,
           2.4442e+01, -4.2321e+00],
         [-3.3906e-01,  1.7553e+00, -7.0530e-01,  ...,  2.5963e+01,
           2.1390e+01, -2.2977e+00],
         [-3.0613e-01, -1.3074e-02, -6.2189e-01,  ...,  1.9722e+01,
           2.0568e+0

/home/alma1/anaconda3/envs/work_env/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:2764: UserWarning: n_quantiles (1000) is greater than the total number of samples (981). n_quantiles is set to n_samples.
  warnings.warn(
/home/alma1/anaconda3/envs/work_env/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:2764: UserWarning: n_quantiles (1000) is greater than the total number of samples (862). n_quantiles is set to n_samples.
  warnings.warn(
/home/alma1/anaconda3/envs/work_env/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:2764: UserWarning: n_quantiles (1000) is greater than the total number of samples (906). n_quantiles is set to n_samples.
  warnings.warn(
/home/alma1/anaconda3/envs/work_env/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:2764: UserWarning: n_quantiles (1000) is greater than the total number of samples (839). n_quantiles is set to n_samples.
  warnings.warn(
Testing:   0%|          | 9/20000 [00:00<26:11, 12.72it/s]/h

...

/home/alma1/anaconda3/envs/work_env/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:2764: UserWarning: n_quantiles (1000) is greater than the total number of samples (891). n_quantiles is set to n_samples.
  warnings.warn(
Testing:   2%|▏         | 367/20000 [00:28<34:51,  9.39it/s]/home/alma1/anaconda3/envs/work_env/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:2764: UserWarning: n_quantiles (1000) is greater than the total number of samples (788). n_quantiles is set to n_samples.
  warnings.warn(
Testing:   2%|▏         | 368/20000 [00:28<36:58,  8.85it/s]

tensor([[[-6.2352e-01,  3.4437e+01, -1.6319e+00,  ...,  7.5818e+01,
           2.0121e+01, -4.0279e+00],
         [-4.4672e-01,  1.0801e+01, -7.2545e-01,  ..., -2.0820e-01,
          -4.9318e-01, -5.3842e-01],
         [-3.9968e-01,  1.1072e+01, -5.8572e-01,  ...,  2.3423e+00,
          -2.6959e-02, -1.1943e+00],
         ...,
         [-2.2408e-01, -7.2353e-01, -1.4112e-01,  ..., -1.2472e-01,
          -4.8996e-01, -8.8641e-01],
         [-2.0617e-01, -6.0470e-01,  2.0478e+00,  ..., -2.8991e-01,
          -3.3021e-01, -7.7162e-01],
         [-2.2695e-01, -6.3214e-01,  3.5538e+00,  ..., -2.9989e-01,
          -3.7261e-01, -8.0850e-01]]], device='cuda:0') shape:  torch.Size([1, 891, 70])
Set: tensor([[[-2.3975e+00,  4.5236e+01,  1.1859e+02,  ...,  1.3097e+02,
          -1.7318e+00, -1.3037e+00],
         [-4.1006e-01, -2.4347e-01,  2.9943e+00,  ..., -6.6439e-01,
          -4.9436e-01, -5.5682e-01],
         [-6.3632e-01, -3.1040e-01,  1.0227e+00,  ...,  7.8784e+00,
          -6.1765e-01

/home/alma1/anaconda3/envs/work_env/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:2764: UserWarning: n_quantiles (1000) is greater than the total number of samples (765). n_quantiles is set to n_samples.
  warnings.warn(
Testing:   2%|▏         | 369/20000 [00:28<36:49,  8.89it/s]/home/alma1/anaconda3/envs/work_env/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:2764: UserWarning: n_quantiles (1000) is greater than the total number of samples (916). n_quantiles is set to n_samples.
  warnings.warn(
Testing:   2%|▏         | 369/20000 [00:28<25:41, 12.74it/s]


tensor([[[ 0.0476, -0.3656, -5.1993, -1.6869, -5.1993],
         [-0.1698, -0.3656, -3.0094, -0.7637, -3.0094],
         [-0.1698, -0.1267, -2.7922, -0.5214, -2.7922],
         ...,
         [-2.2861, -1.6043,  1.2338,  0.5365,  1.3229],
         [-2.2236, -1.6043,  1.2338,  1.0286,  1.3887],
         [-2.1687, -1.6043,  1.1315,  1.7148,  1.4707]]], device='cuda:0') shape:  torch.Size([1, 765, 5])
Set: tensor([[[ 9.7673e+00,  1.8416e-01,  1.4481e+00,  ..., -1.2272e-01,
          -1.6898e-02,  7.8513e+00],
         [ 5.1734e+00, -4.7232e-03,  2.8342e-01,  ..., -6.9639e-02,
          -2.2276e-02,  4.5676e+00],
         [ 5.2940e+00, -5.4092e-03,  1.4593e-01,  ..., -6.7027e-02,
          -1.6449e-02,  4.2557e+00],
         ...,
         [-5.9566e-02, -1.3385e-02, -2.7076e-02,  ...,  6.6932e+00,
          -1.0979e-01, -2.0769e-02],
         [-6.1560e-02, -2.4591e-02, -3.2522e-02,  ...,  6.8324e+00,
          -1.0905e-01, -1.7278e-02],
         [-6.3654e-02, -4.0019e-02, -4.0181e-02,  ..., 

KeyboardInterrupt: 